# YouTube Clipper

<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1cnvPna6JC4nO0vm0Eu5dm4cx2FoRm-kQ?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Executar no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/eduardoboca/YouTubeClipper/blob/main/src/YoutubeClipper.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver código no GitHub</a>
  </td>
</table>

Este projeto amplia as capacidades do Gemini para extrair informações sobre vídeos, fornecendo uma solução adaptável para análise de conteúdo de vídeo em larga escala.

Ele se integra ao YouTube para processar vídeos de qualquer tamanho, mitigando as limitações do Gemini quanto ao tamanho do vídeo e download de vídeos do YouTube.

A aplicação consiste em quatro partes principais:
*   Configurações iniciais: instalação das bibliotecas necessárias e configurações de debug da aplicação.
*   Integração com a API do YouTube para efetuar o download do vídeo e extrair o transcript.
*   Integração com o Gemini para resumir automaticamente as transcrições dos vídeos processados.
*   Exibição do Resultado: Exibição dos resultados processados, incluindo o resumo e a timeline do vídeo.





## Configurações Iniciais

### Modo de Depuração (Debug Mode)

A função `debug_print` foi criada para imprimir mensagens de depuração apenas quando o modo de depuração está ativado. Para ativar o modo de depuração, defina a variável `debug` como `True`.


In [5]:
debug = False

def debug_print(message):
    if debug:
        print(message)

### Passo 1: instalar o SDK do Gemini e as apis de download e transcrição do YouTube

In [6]:
!pip install -q -U google-generativeai

In [7]:
!pip install -q youtube-transcript-api


In [8]:
!pip install -q pytube

## Integração com o YouTube

### Passo 2: fazer o download e a transcrição do vídeo no YouTube



Como exemplo, vamos utilizar o vídeo da aula 4 da imersão em Gemini da Alura

Mas você pode testar com qualquer vídeo! Basta modificar o valor da variável *video_url* no código abaixo com o endereço do seu vídeo

In [9]:
#  URL do vídeo que você quer baixar
video_url = "https://www.youtube.com/watch?v=iwt4bOIHy7s"

from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import json

try:
    yt = YouTube(video_url)
    video_stream = yt.streams.get_highest_resolution()

    # Download do vídeo
    video_stream.download()

    # Extrair o ID do vídeo ID from the URL
    video_id = video_url.split("v=")[1]

    # Transcript do vídeo
    transcripts = YouTubeTranscriptApi.get_transcript(video_id, languages=['pt', 'en'])

    if transcripts:
        subtitle_text = " ".join([entry["text"] for entry in transcripts])

        debug_print("*** Transcripts:")
        debug_print(subtitle_text)
    else:
        debug_print("Transcript não disponível para este vídeo.")

except Exception as e:
    debug_print(f"Erro: {str(e)}")




### Passo 3: dividir o transcript em pedaços menores para poder enviar ao Gemini.

In [10]:
def split_transcript(transcript, max_duration=600):
    parts = []
    current_part = []
    current_duration = 0

    for entry in transcript:
        if current_duration + entry["duration"] <= max_duration:
            # Serializa a entrada para JSON e adiciona à parte atual
            current_part.append(json.dumps(entry))
            current_duration += entry["duration"]
        else:
            # Decodifica cada entrada de transcrição na parte atual
            decoded_part = [bytes(entry, "utf-8").decode("unicode_escape") for entry in current_part]
            parts.append(decoded_part)  # Adiciona a parte atual à lista de partes
            current_part = [json.dumps(entry)]  # Começa uma nova parte com a entrada atual
            current_duration = entry["duration"]

    # Decodifica a última parte
    decoded_part = [bytes(entry, "utf-8").decode("unicode_escape") for entry in current_part]
    parts.append(decoded_part)

    return parts


# Dividir o transcript em partes
segments = split_transcript(transcripts, max_duration=600)

debug_print (f"Transcript dividido em {len(segments)} partes")
for segment in segments:
  debug_print (f"{segment[:1000]}")
  debug_print ("---")


## Integração com o Gemini




### Passo 4: configurar a API Key do Gemini






In [11]:
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

### Passo 5: configurar o modelo

In [12]:
generation_config = {
    "candidate_count" : 1,
    "temperature" : 0.5
}

safety_settings = {
    "HARASSMENT" : "BLOCK_NONE",
    "HATE" : "BLOCK_NONE",
    "SEXUAL" : "BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE"
}

model = genai.GenerativeModel("gemini-1.0-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

In [13]:
prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
prompt += "\n".join(segments[0])

debug_print(prompt)

### Passo 6: solicitar o resumo ao Gemini
Primeiro vamos solicitar o resumo e a timeline de cada trecho


In [14]:
summaries = []
clips = []

for segment in segments:
  prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
  prompt += " Aqui está o conteúdo:"
  prompt += "\n".join(segment)
  prompt += " Os dados do vídeo estão no formato JSON {text:'', start:'', duration:''}. start representa o início do trecho em segundos, e duration representa a duração do trecho em segundos;"
  prompt += " Quais são os assuntos discutidos nesse vídeo? Crie uma lista contendo uma descrição do assunto e o valor do campo start em que se inicia o assunto."
  prompt += " Também preciso que você faça um breve resumo do vídeo inteiro."
  prompt += " Responda no formato JSON, de forma que uma aplicação possa fazer o parse da resposta. Os campos do JSON devem ser exatamente estes: 1. summary: resumo do vídeo e 2: topics: lista dos assuntos, que também deve ter um formato JSON assim definido: 1. description e 2.start"
  prompt += " O valor de start deve ser exatamente igual ao encontrado no json original."
  response = model.generate_content(prompt)
  response_text = response.text
  response_text = response_text.replace("```json", '')
  response_text = response_text.replace("```", '')

  json_object = json.loads(response_text)

  summaries.append(json_object["summary"])
  clips.extend(json_object["topics"])

for clip in clips:
    debug_print(clip)


### Passo 7: juntar todos os trechos e montar o resumo geral

In [15]:
summaries_string = '\n'.join(summaries)

prompt = "Preciso descrever o conteúdo de um vídeo do YouTube."
prompt += " O vídeo contém vários trechos. Vou enviar o resumo de cada trecho. Você deve ler todos eles e gerar um resumo geral."
prompt += " Retorne apenas o resumo geral. Estes são os resumos:\n"

prompt += summaries_string
response = model.generate_content(prompt)

total_summary = response.text


## E finalmente temos o resumo do vídeo e a timeline!

In [25]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

display (to_markdown("### Resumo:"))
display (to_markdown(total_summary))
display (to_markdown("### Timeline:"))

for clip in clips:
  if isinstance(clip['start'], str):
    clip['start'] = clip['start'].replace("'", "")
    clip['start'] = float(clip['start'])

for clip in clips:
    link = f"{video_url}&t={clip['start']}s"
    hours = int(clip['start'] // 3600)
    minutes = int((clip['start'] % 3600) // 60)
    seconds = int(clip['start'] % 60)

    # Formato do tempo HH:MM:SS
    time_formatted = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    #time_formatted = clip['start']
    start_aligned = f"{time_formatted}".ljust(10)
    description_aligned = clip['description'].ljust(50)
    display (to_markdown(f"{start_aligned} - [{description_aligned}]({link})"))



> ### Resumo:

> Este vídeo é uma aula prática sobre o desenvolvimento com o Gemini, o LLM do Google. A aula aborda a integração da API Gemini com Python para criar chatbots, explicando modelos, parâmetros de geração e segurança. Ele também oferece um tutorial para criar um chatbot interativo e um desafio para os alunos criarem projetos usando a API Gemini. O vídeo orienta os alunos sobre como enviar projetos para o GitHub e enfatiza as diretrizes éticas do Google.

> ### Timeline:

> 00:00:01   - [Boas-vindas e introdução à aula                   ](https://www.youtube.com/watch?v=iwt4bOIHy7s&t=1.298s)

> 00:00:07   - [Apresentação dos instrutores                      ](https://www.youtube.com/watch?v=iwt4bOIHy7s&t=7.0s)

...

> 01:06:48   - [Upload de arquivos do Colab para o GitHub         ](https://www.youtube.com/watch?v=iwt4bOIHy7s&t=4008.58s)

> 01:10:11   - [Importância do ReadMe File para conexão com o Colab](https://www.youtube.com/watch?v=iwt4bOIHy7s&t=4211.672s)

> 01:12:43   - [Markdown para documentação de projetos            ](https://www.youtube.com/watch?v=iwt4bOIHy7s&t=4363.0s)

> 01:13:27   - [Upload de projetos para o GitHub usando o Colab   ](https://www.youtube.com/watch?v=iwt4bOIHy7s&t=4407.756s)

> 01:19:45   - [Ocultar API Key usando segredos do Colab          ](https://www.youtube.com/watch?v=iwt4bOIHy7s&t=4785.0s)